In [1]:
import plotly.io as pio
import plotly.express as px
import plotly.subplots as pss
import plotly.graph_objects as go
pio.templates.default = "plotly_white"
COLOUR_CONTINIOUS_SCALE="bluered"

# The one below should be both color-blind and print friendly. It's based on: https://colorbrewer2.org/#type=sequential&scheme=RdPu&n=5
COLOUR_ARRAY = ["#feebe2", "#fbb4b9", "#f768a1", "#c51b8a", "#7a0177"]


In [2]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [3]:
from scipy import stats
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import pandas as pd

In [4]:
from collections import defaultdict
import csv

In [5]:
from delitoolkit.delidata import DeliData

In [6]:
d = DeliData().corpus

In [7]:
allowed = {
    'vowels': {'A', 'O', 'U', 'E', 'I', 'Y'},
    'consonants': {'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'X', 'Z', 'W'},
    'odds': {'1', '3', '5', '7', '9'},
    'evens': {'0', '2', '4', '6', '8'}
}

def process_solution_simple(solution):
    res = set()
    for item in solution:
        for a_k, a_v in allowed.items():
            if item in a_v:
                res.update(a_k[0])
    return "".join(sorted(list(res)))

In [8]:
exclude_9_correct = []

In [9]:
from collections import Counter
all_sols_counter = Counter()

In [10]:
import json

In [11]:
list_of_keys = sorted(list(d.keys()))
# exclude_9_correct = ["ec8896c6-56fb-4187-a932-92ac6bbd6f49", "c47d75d6-98f9-4a03-a0b1-54b341849ffc", "e731d8d4-d6f3-44ca-9d04-0d2caa2f0111", "02972b77-c909-499e-8d4b-8920a25707f0", "b1d5c668-eb55-440a-ade6-155784bcd0a4", "28e38a3c-0298-4585-b05f-cef4968f56c1", "f052602b-0dbe-45f8-8407-f0887616ad41", "dca3b7c4-9483-4d9d-b76b-b7244cd8529a", "513c227d-c2c2-4eb6-8730-feb1d513cbc1"]

In [12]:
all_transitions = defaultdict(lambda: defaultdict(lambda: 0))
bi_transitions = defaultdict(lambda: defaultdict(lambda: 0))
all_sols = 0
in_count = 0

for room_id in list_of_keys:
    if room_id in exclude_9_correct:
        continue
    room = d[room_id]
    sol_transition = []
    for room_message in room:
        if room_message['message_type'] == 'MESSAGE' and len(room_message['sol_tracker_message']) != 0:
            key_sols = process_solution_simple(room_message['sol_tracker_message'].split(','))
            sol_transition.append(key_sols)
    all_sols_counter.update(sol_transition)
    if len(sol_transition) < 3:
        all_transitions[room_id][">".join(sol_transition)] = 0
    else:
        grams = [sol_transition[i: i + 3] for i in range(len(sol_transition) - 3 + 1)]
        for prev, curr, nxt in grams:
            in_count += 1
            all_transitions[room_id][prev + ">" + curr + ">" + nxt] += 1
    if len(sol_transition) < 3:
        pass
    else:
        grams = [sol_transition[i: i + 2] for i in range(len(sol_transition) - 2 + 1)]
        for prev, nxt in grams:
            bi_transitions[room_id][prev + ">" + nxt] += 1

In [13]:
all_sols_counter.most_common(5)

[('v', 1217), ('ceov', 1036), ('ev', 990), ('ov', 788), ('e', 596)]

In [14]:
all_transitions

defaultdict(<function __main__.<lambda>()>,
            {'00639b9b-3a45-4c59-90f6-a4738de92fd4': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'v>o>c': 1,
                          'o>c>v': 1,
                          'c>v>c': 1,
                          'v>c>ev': 1,
                          'c>ev>ov': 1,
                          'ev>ov>ev': 1,
                          'ov>ev>v': 1,
                          'ev>v>v': 1,
                          'v>v>ov': 1,
                          'v>ov>e': 1,
                          'ov>e>ov': 1,
                          'e>ov>ev': 1,
                          'ov>ev>e': 1,
                          'ev>e>o': 1,
                          'e>o>e': 1,
                          'o>e>e': 1}),
             '0128a2aa-53a2-4874-8bfe-b71ce287e7c2': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'e>ov>ev': 1,
                          'ov>ev>v': 1,
                  

In [15]:
room_to_performancegain = {}
message_count = {}
with open('meta_stats.tsv', 'r') as rf:
    tsv_reader = csv.reader(rf, delimiter='\t')
    next(tsv_reader)
    
    for item in tsv_reader:
        room_to_performancegain[item[0]] = float(item[-6])
        message_count[item[0]] = float(item[2])


# ordered = sorted(room_to_performancegain, key=room_to_performancegain.get, reverse=True)

positive_data = {}
negative_data = {}
neutral_data = {}
positive_bi = {}
negative_bi = {}
for key, item in all_transitions.items():
    if room_to_performancegain[key] > 0:
        positive_data[key] = item
        positive_bi[key] = item
    elif room_to_performancegain[key] < 0:
        negative_data[key] = item
        negative_bi[key] = item
    else:
        neutral_data[key] = item
    
for key, item in bi_transitions.items():
    if room_to_performancegain[key] > 0:
        positive_bi[key] = item
    elif room_to_performancegain[key] < 0:
        negative_bi[key] = item
    


In [16]:
def total_unique_normalised_count(collection, total_count=1):
    if len(collection) == 1 and len(collection[0][0]) < 3:
        return 0
    return len(collection)/total_count

def total_normalised_count(collection, total_count=1):
    if len(collection) == 1 and len(collection[0][0]) < 3:
        return 0
    total = sum([c for _, c in collection])
    return total / total_count

def repetitions(collection, total_count=1):
    rep = 0
    if len(collection) == 1 and len(collection[0][0]) < 3:
        return 0
    for (f, tt, t), count in collection:
        if len(set([f, tt, t])) == 1:
            rep += count
    return rep / total_count

def circular(collection, total_count=1):
    rep = 0
    if len(collection) == 1 and len(collection[0][0]) < 3:
        return 0
    for (f, tt, t), count in collection:
        if f == t and f != tt:
            rep += count
            
    return rep / total_count

def total_unique_solutions(collection, total_count=1):
    rep = 0
    unique = set()
    if len(collection) == 1 and len(collection[0][0][0]) == 0:
        return 0
    elif len(collection) == 1 and len(collection[0][0]) < 3:
        unique.update(collection[0][0])
        return len(unique) / total_count
    for (f, tt, t), count in collection:
        unique.add(f)
        unique.add(tt)        
        unique.add(t)
    return len(unique) / total_count



In [17]:
def get_probings_byroom(room_id, corpus):
    room = corpus[room_id]
    count_m, count_s, count_r = 0, 0, 0
    for room_message in room:
        if room_message['message_type'] == 'MESSAGE' and room_message['annotation_type'] == 'Probing':
            if room_message['annotation_target'] == 'Moderation':
                count_m += 1
            elif room_message['annotation_target'] == 'Reasoning':
                count_r += 1
            else:
                count_s += 1
            
    return count_m, count_s, count_r

def count_initials(room_id, corpus):
    obj = json.loads(corpus[room_id][0]['sol_tracker_all'])
    unique = set()
    count = 0
    for k, v in obj.items():
        unique.add("".join(v))
        count += 1
    
    return len(unique), count

In [18]:
positive_result = []
negative_result = []
neutral_result = []
for room, items in positive_data.items():
    states = []
    for k, i in items.items():
        states.append((k.split('>'), i))
#     m_c = message_count[room]
    m_c = 1 
    positive_result.append({'room_id': room, 'pg': room_to_performancegain[room],
                            'all_transitions': total_normalised_count(states, m_c),
                            'unique_transitions': total_unique_normalised_count(states, m_c),
                            'repetitions': repetitions(states, m_c),
                            'circular': circular(states, m_c),
                            'total_unique_sols': total_unique_solutions(states, m_c),
                            'message_count': message_count[room],
                            'probing_mod': get_probings_byroom(room, d)[0],
                            'probing_sol': get_probings_byroom(room, d)[1],
                            'probing_res': get_probings_byroom(room, d)[2],
                            'initial': count_initials(room, d)[0],
                            'participant_count': count_initials(room, d)[1]
                           })
    
for room, items in negative_data.items():
    states = []
    for k, i in items.items():
        states.append((k.split('>'), i))
#     m_c = message_count[room]
    m_c = 1 
    negative_result.append({'room_id': room, 'pg': room_to_performancegain[room],
                            'all_transitions': total_normalised_count(states, m_c),
                            'unique_transitions': total_unique_normalised_count(states, m_c),
                            'repetitions': repetitions(states, m_c),
                            'circular': circular(states, m_c),
                            'total_unique_sols': total_unique_solutions(states, m_c),
                            'message_count': message_count[room],
                            'probing_mod': get_probings_byroom(room, d)[0],
                            'probing_sol': get_probings_byroom(room, d)[1],
                            'probing_res': get_probings_byroom(room, d)[2],
                             'initial': count_initials(room, d)[0],
                            'participant_count': count_initials(room, d)[1]
                           })

    
for room, items in neutral_data.items():
    states = []
    for k, i in items.items():
        states.append((k.split('>'), i))
#     m_c = message_count[room]
    m_c = 1 
    neutral_result.append({'room_id': room, 'pg': room_to_performancegain[room],
                            'all_transitions': total_normalised_count(states, m_c),
                            'unique_transitions': total_unique_normalised_count(states, m_c),
                            'repetitions': repetitions(states, m_c),
                            'circular': circular(states, m_c),
                            'total_unique_sols': total_unique_solutions(states, m_c),
                            'message_count': message_count[room],
                            'probing_mod': get_probings_byroom(room, d)[0],
                            'probing_sol': get_probings_byroom(room, d)[1],
                            'probing_res': get_probings_byroom(room, d)[2],
                            'initial': count_initials(room, d)[0],
                            'participant_count': count_initials(room, d)[1]
                           })


In [19]:
print(len(positive_result), len(negative_result), len(neutral_result))

318 82 100


# Predictive power

In [20]:
merged_data = [*positive_result, *negative_result, *neutral_result]
df = pd.DataFrame(merged_data)

In [21]:
df

,room_id,pg,all_transitions,unique_transitions,repetitions,circular,total_unique_sols,message_count,probing_mod,probing_sol,probing_res,initial,participant_count
0,0128a2aa-53a2-4874-8bfe-b71ce287e7c2,0.187500,26.0,26.0,0.0,4.0,7.0,64.0,0,0,2,3,4
1,01e7d47c-8831-4492-abb0-88f94c7b02a3,0.500000,16.0,16.0,1.0,3.0,8.0,30.0,1,0,4,3,5
2,02972b77-c909-499e-8d4b-8920a25707f0,0.083333,15.0,12.0,1.0,5.0,6.0,54.0,2,5,0,3,4
3,03714d9c-9c47-4103-9b61-ed51c76e1ef6,0.166667,10.0,10.0,1.0,4.0,4.0,18.0,0,0,3,4,4
4,038c3e5e-35bf-4e26-b96d-8ff92bb8c830,0.250000,5.0,5.0,0.0,2.0,4.0,21.0,2,1,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,f241e2fb-4743-4e66-9152-f1103b54073f,0.000000,0.0,0.0,0.0,0.0,2.0,15.0,1,0,1,2,4
496,f2e7ae3f-ca02-4896-b0bf-b45caf4b8479,0.000000,7.0,7.0,0.0,2.0,4.0,15.0,0,0,1,4,4
497,f8704b44-06e5-4938-8c61-6a64aabf442b,0.000000,13.0,13.0,0.0,3.0,6.0,31.0,7,1,0,2,4
498,fb18014b-0af6-4ad4-a0d1-5d0458780d67,0.000000,24.0,21.0,1.0,3.0,7.0,78.0,5,0,3,2,3


In [22]:
df.describe()

,pg,all_transitions,unique_transitions,repetitions,circular,total_unique_sols,message_count,probing_mod,probing_sol,probing_res,initial,participant_count
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.126700,10.566000,9.894000,0.860000,1.466000,5.124000,28.006000,1.570000,0.980000,0.926000,3.090000,3.948000
std,0.195140,8.092009,7.541652,1.369727,1.677639,2.230375,15.161455,1.476978,1.187611,1.293007,0.929315,1.023421
min,-0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,1.000000,2.000000
25%,0.000000,5.000000,5.000000,0.000000,0.000000,3.000000,18.000000,0.000000,0.000000,0.000000,2.000000,3.000000
50%,0.125000,9.000000,8.000000,0.000000,1.000000,5.000000,25.000000,1.000000,1.000000,1.000000,3.000000,4.000000
75%,0.250000,15.000000,14.000000,1.000000,2.000000,7.000000,34.000000,2.000000,1.000000,1.000000,4.000000,5.000000
max,0.750000,54.000000,50.000000,10.000000,10.000000,11.000000,104.000000,8.000000,7.000000,9.000000,6.000000,7.000000


In [23]:
# 0.126700	10.596000	9.924000	0.860000	1.466000	5.124000	28.006000

In [24]:
from scipy.stats import zscore

In [25]:
df[['all_transitions']] = df[['all_transitions']].transform(lambda x: zscore(x))
df[['circular']] = df[['circular']].transform(lambda x: zscore(x))
df[['message_count']] = df[['message_count']].transform(lambda x: zscore(x))
df[['repetitions']] = df[['repetitions']].transform(lambda x: zscore(x))
df[['total_unique_sols']] = df[['total_unique_sols']].transform(lambda x: zscore(x))
df[['unique_transitions']] = df[['unique_transitions']].transform(lambda x: zscore(x))
df[['probing_mod']] = df[['probing_mod']].transform(lambda x: zscore(x))
df[['probing_res']] = df[['probing_res']].transform(lambda x: zscore(x))
df[['probing_sol']] = df[['probing_sol']].transform(lambda x: zscore(x))

df[['pg']] = df[['pg']].transform(lambda x: zscore(x))


In [26]:
len(df[df['circular'] > 0])

186

In [27]:
all_trans = ols("pg ~ total_unique_sols  + unique_transitions + circular + repetitions + message_count ", data=df).fit()
all_trans.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     pg   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     11.90
Date:                Thu, 01 Feb 2024   Prob (F-statistic):           6.91e-11
Time:                        20:38:28   Log-Likelihood:                -681.04
No. Observations:                 500   AIC:                             1374.
Df Residuals:                     494   BIC:                             1399.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -5.378e-17      0.043  -1.27e-15      1.000      -0.084       0.084
total_unique_sols      0.3162      0.078      4.073      0.000       0.164       0.469
unique_transitions     0.0375      0.123      0.305      0.761      -0.204       0.279
circular               0.1770      0.062      2.854      0.004       0.055       0.299
repetitions            0.0439      0.047      0.927      0.354      -0.049       0.137
message_count         -0.2352      0.081     -2.921      0.004      -0.393      -0.077
==============================================================================
Omnibus:                       12.550   Durbin-Watson:                   0.825
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               21.376
Skew:                          -0.135   Prob(JB):                     2.28e-05
Kurtosis:                       3.976   Cond. No.                         6.10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Effect of probing on diversity

In [28]:
merged_data = [*positive_result, *negative_result, *neutral_result]
df_raw = pd.DataFrame(merged_data)

In [29]:
df_reduced = df_raw[df_raw['participant_count'] >= 3]
df_reduced = df_raw[df_raw['initial'] <= 2]
print(len(df_reduced))
df_reduced['probing'] = df_reduced['probing_sol'] + df_reduced['probing_mod'] + df_reduced['probing_res']


135


/var/folders/vp/7bwf0dxs2rd7_0tmxn77vx540000gn/T/ipykernel_69159/3244310022.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
df_reduced = df_reduced.drop('room_id', axis=1)


In [31]:
df_reduced.describe()

,pg,all_transitions,unique_transitions,repetitions,circular,total_unique_sols,message_count,probing_mod,probing_sol,probing_res,initial,participant_count,probing
count,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000
mean,0.085710,8.607407,8.022222,0.814815,1.229630,4.644444,26.170370,1.466667,0.755556,0.711111,1.925926,3.111111,2.933333
std,0.201278,6.862287,6.361576,1.383189,1.583218,2.159556,14.754984,1.520016,1.003477,1.183636,0.262867,0.903283,2.369583
min,-0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000
25%,0.000000,4.000000,4.000000,0.000000,0.000000,3.000000,17.000000,0.000000,0.000000,0.000000,2.000000,2.000000,1.000000
50%,0.062500,7.000000,7.000000,0.000000,1.000000,4.000000,22.000000,1.000000,1.000000,0.000000,2.000000,3.000000,2.000000
75%,0.187500,12.000000,11.500000,1.000000,2.000000,6.000000,33.000000,2.000000,1.000000,1.000000,2.000000,4.000000,4.000000
max,0.583333,42.000000,41.000000,7.000000,10.000000,11.000000,86.000000,8.000000,7.000000,7.000000,2.000000,5.000000,11.000000


In [32]:
df_reduced.corr() # 0.328990 0.424303

,pg,all_transitions,unique_transitions,repetitions,circular,total_unique_sols,message_count,probing_mod,probing_sol,probing_res,initial,participant_count,probing
pg,1.000000,0.271750,0.272572,0.060564,0.232166,0.334456,0.138371,-0.079471,0.009518,0.220606,0.015106,0.030861,0.063248
all_transitions,0.271750,1.000000,0.984343,0.441216,0.687005,0.742344,0.832040,0.203712,0.259058,0.337822,0.186474,0.280383,0.409128
unique_transitions,0.272572,0.984343,1.000000,0.318510,0.687093,0.776280,0.826806,0.208067,0.291944,0.336838,0.175036,0.277487,0.425356
repetitions,0.060564,0.441216,0.318510,1.000000,0.114981,0.112702,0.358440,0.034312,-0.048987,0.117501,0.146714,0.159943,0.059958
circular,0.232166,0.687005,0.687093,0.114981,1.000000,0.340547,0.562798,0.110190,0.275157,0.230798,0.059108,0.169885,0.302494
total_unique_sols,0.334456,0.742344,0.776280,0.112702,0.340547,1.000000,0.592809,0.137316,0.279857,0.271905,0.111011,0.112219,0.342418
message_count,0.138371,0.832040,0.826806,0.358440,0.562798,0.592809,1.000000,0.307210,0.243756,0.444673,0.120646,0.203503,0.522412
probing_mod,-0.079471,0.203712,0.208067,0.034312,0.110190,0.137316,0.307210,1.000000,0.080239,0.058900,0.031129,0.168494,0.704871
probing_sol,0.009518,0.259058,0.291944,-0.048987,0.275157,0.279857,0.243756,0.080239,1.000000,0.185140,0.044009,0.021955,0.567433
probing_res,0.220606,0.337822,0.336838,0.117501,0.230798,0.271905,0.444673,0.058900,0.185140,1.000000,0.050635,-0.116333,0.615699


In [33]:
import plotly.express as px


In [34]:
scatter = px.scatter(df_reduced,
                     x="total_unique_sols",
                     y="probing",
                     trendline="ols",
                     color_discrete_sequence=[COLOUR_ARRAY[3]])
fig = go.Figure(scatter)
fig.add_annotation(text='R-squared: 0.117<br>Correlation: 0.34', 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=1.06,
                    y=1,
                    bordercolor=COLOUR_ARRAY[3],
                    borderwidth=2, font = {'size':20})

fig.update_layout(
    xaxis_title="Number of unique solutions discussed", yaxis_title="Number of probing utterances"
)
fig.update_layout(font_size=20)
# fig.update_layout(width=1200)
fig.update_traces(marker=dict(size=10,
                              line=dict(width=2,
                                        )),
                  selector=dict(mode='markers'))

fig.show()


results = px.get_trendline_results(scatter)
print(results)

results.px_fit_results.iloc[0].summary()

                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     17.67
Date:                Thu, 01 Feb 2024   Prob (F-statistic):           4.80e-05
Time:                        20:38:30   Log-Likelihood:                -299.10
No. Observations:                 135   AIC:                             602.2
Df Residuals:                     133   BIC:                             608.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1883      0.458      2.597      0.010       0.283       2.093
x1             0.3757      0.089      4.203      0.000       0.199       0.553
==============================================================================
Omnibus:                       31.403   Durbin-Watson:                   2.196
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.994
Skew:                           1.164   Prob(JB):                     3.79e-11
Kurtosis:                       4.764   Cond. No.                         12.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
import plotly.io as pio

# Set the scale factor for resolution
# scale=1 corresponds to 96 dpi, scale=2 corresponds to 192 dpi, scale=3 corresponds to 288 dpi, and so on.
scale = 6  # Adjust this to control the resolution

filename = "probing_uniquesols.png"
pio.write_image(fig, filename, format='png', scale=scale)

In [36]:
all_trans = ols("total_unique_sols ~ probing_mod + probing_res + probing_sol ", data=df_reduced).fit()
all_trans.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      total_unique_sols   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     7.083
Date:                Thu, 01 Feb 2024   Prob (F-statistic):           0.000190
Time:                        20:38:32   Log-Likelihood:                -284.85
No. Observations:                 135   AIC:                             577.7
Df Residuals:                     131   BIC:                             589.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.7606      0.280     13.411      0.000       3.206       4.315
probing_mod     0.1502      0.116      1.299      0.196      -0.079       0.379
probing_res     0.4070      0.151      2.702      0.008       0.109       0.705
probing_sol     0.4951      0.178      2.783      0.006       0.143       0.847
==============================================================================
Omnibus:                        0.902   Durbin-Watson:                   1.700
Prob(Omnibus):                  0.637   Jarque-Bera (JB):                1.019
Skew:                           0.161   Prob(JB):                        0.601
Kurtosis:                       2.722   Cond. No.                         4.23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""